In [52]:
#@title First load the data! (Click Play)
import numpy as np
from sklearn.preprocessing import StandardScaler
!wget -nc http://www.fredjo.com/files/ihdp_npci_1-100.train.npz
!wget -nc http://www.fredjo.com/files/ihdp_npci_1-100.test.npz 

def load_IHDP_data(training_data,testing_data,i):
    with open(training_data,'rb') as trf, open(testing_data,'rb') as tef:
        train_data=np.load(trf); test_data=np.load(tef)
        y=np.concatenate(   (train_data['yf'][:,i],   test_data['yf'][:,i])).astype('float32') #most GPUs only compute 32-bit floats
        t=np.concatenate(   (train_data['t'][:,i],    test_data['t'][:,i])).astype('float32')
        x=np.concatenate(   (train_data['x'][:,:,i],  test_data['x'][:,:,i]),axis=0).astype('float32')
        mu_0=np.concatenate((train_data['mu0'][:,i],  test_data['mu0'][:,i])).astype('float32')
        mu_1=np.concatenate((train_data['mu1'][:,i],  test_data['mu1'][:,i])).astype('float32')

        data={'x':x,'t':t,'y':y,'t':t,'mu_0':mu_0,'mu_1':mu_1}
        data['t']=data['t'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension 
        data['y']=data['y'].reshape(-1,1)
        
        #rescaling y between 0 and 1 often makes training of DL regressors easier
        data['y_scaler'] = StandardScaler().fit(data['y'])
        data['ys'] = data['y_scaler'].transform(data['y'])

    return data

ind = 7
rep = 5
data = load_IHDP_data(training_data='./ihdp_npci_1-100.train.npz',testing_data='./ihdp_npci_1-100.test.npz',i = ind)
for key in data:
    if key != 'y_scaler':
        data[key] = np.repeat(data[key],repeats = rep, axis = 0)
np.shape(data['x'])


文件 “ihdp_npci_1-100.train.npz” 已经存在；不获取。

文件 “ihdp_npci_1-100.test.npz” 已经存在；不获取。



(3735, 25)

In [53]:
import tensorflow as tf
import numpy as np
import datetime
#Colab command to allow us to run Colab in TF2
!rm -rf ./logs/ 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%reload_ext tensorboard 

In [54]:
from tensorflow.keras.callbacks import Callback

def pdist2sq(A, B):
    #helper for PEHEnn
    #calculates squared euclidean distance between rows of two matrices  
    #https://gist.github.com/mbsariyildiz/34cdc26afb630e8cae079048eef91865
    # squared norms of each row in A and B
    na = tf.reduce_sum(tf.square(A), 1)
    nb = tf.reduce_sum(tf.square(B), 1)    
    # na as a row and nb as a column vectors
    na = tf.reshape(na, [-1, 1])
    nb = tf.reshape(nb, [1, -1])
    # return pairwise euclidean difference matrix
    D=tf.reduce_sum((tf.expand_dims(A, 1)-tf.expand_dims(B, 0))**2,2) 
    return D

from evaluation import Full_Metrics, metrics_for_cevae

In [55]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model

import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.stats import sem
import tensorflow.keras as tfk
import tensorflow_probability as tfp
import tensorflow.keras.layers as tfkl
tfd,tfpl = tfp.distributions,tfp.layers
import tensorflow.keras.backend as tfkb

class tarnet(tf.keras.Model):
    def __init__(self, input_dim, reg_l2):
        super(tarnet, self).__init__()
        ########################################
        # networks
        self.activation = 'elu'
        self.share_bottom = tfk.Sequential(
            [
                tfkl.InputLayer([input_dim]),
                tfkl.Dense(100,activation = self.activation, kernel_initializer='RandomNormal'),
                tfkl.Dense(100,activation = self.activation, kernel_initializer='RandomNormal'),
                tfkl.Dense(100,activation = self.activation, kernel_initializer='RandomNormal'),
            ])
        self.tower_t0 = tfk.Sequential(
            [
                tfkl.InputLayer([100]),
                tfkl.Dense(100,activation = self.activation, kernel_regularizer=regularizers.l2(reg_l2)),
                tfkl.Dense(100,activation = self.activation, kernel_regularizer=regularizers.l2(reg_l2)),
                tfkl.Dense(1,activation = None, kernel_regularizer=regularizers.l2(reg_l2)),
            ])

        self.tower_t1 = tfk.Sequential(
            [
                tfkl.InputLayer([100]),
                tfkl.Dense(100,activation = self.activation, kernel_regularizer=regularizers.l2(reg_l2)),
                tfkl.Dense(100,activation = self.activation, kernel_regularizer=regularizers.l2(reg_l2)),
                tfkl.Dense(1,activation = None, kernel_regularizer=regularizers.l2(reg_l2)),
            ])
    def call(self, data, training=False, serving=False):
        # Dataset_inp
        hidden_bottom = self.share_bottom(data)
        y_t0 = self.tower_t0(hidden_bottom)
        y_t1 = self.tower_t1(hidden_bottom)
    
        output = tf.concat([y_t0,y_t1,hidden_bottom],-1)
        return output

#make model
tarnet_model=tarnet(data['x'].shape[1],.01)
# fake_inputs = tfk.Input(25,dtype = tf.float32)
# tarnet_model(fake_inputs)

In [58]:
# every loss function in TF2 takes 2 arguments, a vector of true values and a vector predictions
def regression_loss(concat_true, concat_pred):
    #computes a standard MSE loss for TARNet
    y_true = concat_true[:, 0] #get individual vectors
    t_true = concat_true[:, 1]
 
    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]
 
    #Each head outputs a prediction for both potential outcomes
    #We use t_true as a switch to only calculate the factual loss
    loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - y0_pred))
    loss1 = tf.reduce_sum(t_true * tf.square(y_true - y1_pred))
    #note Shi uses tf.reduce_sum for her losses even though mathematically we should be using the mean
    #tf.reduce_mean and tf.reduce_sum should be equivalent, but maybe having larger error gradients makes training easier?
    return loss0 + loss1
 
### MAIN CODE ####
 
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD
 
val_split=0.2
batch_size=128
verbose=True
i = 0
tf.random.set_seed(i)
np.random.seed(i)
yt = np.concatenate([data['ys'], data['t']], 1) #we'll use both y and t to compute the loss
 
# Clear any logs from previous runs
!rm -rf ./logs/ 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
 
sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=40, min_delta=0), 
        #40 is Shi's recommendation patience for this dataset, but you should tune for your data 
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0, cooldown=0, min_lr=0),
        #This learning rate scheduling is quite agressive which seems good for this dataset
        Full_Metrics(data,verbose),
        tensorboard_callback
    ]
#optimizer hyperparameters
sgd_lr = 1e-5
momentum = 0.9
tarnet_model.compile(optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True),
                    loss=regression_loss,
                    metrics=regression_loss)
 
tarnet_model.fit(x=data['x'],y=yt,
                callbacks=sgd_callbacks,
                validation_split=val_split,
                epochs=300,
                batch_size=batch_size,
                verbose=verbose)
print("Done!")

Epoch 1/300


/usr/local/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


24/24 [==============================] - 2s 63ms/step - loss: 21.7563 - regression_loss: 17.5126 - val_loss: 28.7704 - val_regression_loss: 25.0095 - lr: 1.0000e-05
Epoch 2/300
24/24 [==============================] - 1s 49ms/step - loss: 21.7472 - regression_loss: 17.4976 - val_loss: 28.8649 - val_regression_loss: 25.1071 - lr: 1.0000e-05
Epoch 3/300
24/24 [==============================] - 1s 54ms/step - loss: 21.8144 - regression_loss: 17.5501 - val_loss: 29.2426 - val_regression_loss: 25.4964 - lr: 1.0000e-05
Epoch 4/300
24/24 [==============================] - 1s 51ms/step - loss: 21.7342 - regression_loss: 17.4572 - val_loss: 28.9093 - val_regression_loss: 25.1588 - lr: 1.0000e-05
Epoch 5/300
24/24 [==============================] - 1s 52ms/step - loss: 21.6962 - regression_loss: 17.4014 - val_loss: 28.9070 - val_regression_loss: 25.1570 - lr: 1.0000e-05
Epoch 6/300
24/24 [==============================] - 1s 46ms/step - loss: 21.7094 - regression_loss: 17.4572 - val_loss: 29.120

In [59]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 11635), started 1 day, 0:29:34 ago. (Use '!kill 11635' to kill it.)